In [2]:
import pandas as pd
import requests
import time
import aylien_news_api
from aylien_news_api.rest import ApiException

In [3]:
df = pd.read_csv("uci-news-aggregator.csv")
fin_df1 = pd.read_csv("pulse-news-dump.csv", header = 0, dtype = {"TITLE":object,"URL":object}, usecols = [0,1])

In [4]:
fin_df2 = fin_df1.sample(frac = 0.26)

In [5]:
cat = []
for i in range(len(fin_df2)):
    cat.append("b")

In [6]:
fin_df2["CATEGORY"] = cat 

In [7]:
res = df.append(fin_df2)

In [8]:
res.dropna(subset = ["TITLE","URL"], inplace = True)

In [12]:
aylien_news_api.configuration.api_key['X-AYLIEN-NewsAPI-Application-ID'] = '98f3fe2d'
aylien_news_api.configuration.api_key['X-AYLIEN-NewsAPI-Application-Key'] = '8250b008cf5d2df152adc1a886b52924'
api_instance = aylien_news_api.DefaultApi()

In [13]:
def fetch_new_stories(params={}):
    fetched_stories = []
    stories = None
    while stories is None or len(stories) > 0:
        try:
            response = api_instance.list_stories(**params)
        except ApiException as e:
            if (e.status == 429):
                print('Usage limit are exceeded. Waiting for 60 seconds...')
                time.sleep(60)
                continue

        stories = response.stories
        params['cursor'] = response.next_page_cursor
        fetched_stories += stories
    return fetched_stories

In [14]:
dic1 = {"title":"ipl","language":["en"],"categories_taxonomy":"iab-qag","categories_id":["IAB17"],"source_locations_country": ["IN"],'sort_by': 'social_shares_count.facebook','published_at_start': 'NOW-30DAYS','published_at_end': 'NOW',"per_page":100}
indian_sports_stories = fetch_new_stories(dic1)

In [33]:
headline = []
url = []
category = []
for story in indian_sports_stories:
    headline.append(story.title)
    url.append(story.links.permalink)
    category.append("e")

In [34]:
sports_df = pd.DataFrame(data = {"TITLE":headline,"URL":url,"CATEGORY":category})

In [35]:
res1 = res.append(sports_df)

In [38]:
res1.to_csv("/home/harish/final_project/Final_dataset.csv")